<a href="https://colab.research.google.com/github/abdalganih1/readOnly/blob/main/RADDet_Implementation_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# تنفيذ مشروع RADDet على Google Colab

هذا الدفتر يشرح خطوات تنفيذ مشروع RADDet بالكامل على بيئة Google Colab، بدءًا من تحميل المشروع والبيانات، وصولاً إلى تدريب النموذج وتقييمه.

### الخطوة الأولى: نسخ مستودع المشروع

أولاً، نقوم بنسخ مستودع المشروع من GitHub والانتقال إلى مجلد المشروع.

In [7]:
# Cell 1: Clone the repository and move into its directory
!git clone https://github.com/ZhangAoCanada/RADDet.git
%cd RADDet

Cloning into 'RADDet'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 315 (delta 69), reused 70 (delta 65), pack-reused 232 (from 1)
Receiving objects: 100% (315/315), 85.81 MiB | 25.27 MiB/s, done.
Resolving deltas: 100% (158/158), done.
/content/RADDet/RADDet


### الخطوة الثانية: تعديل وتثبيت المكتبات المطلوبة (مصحح)

ملف `requirements.txt` الأصلي قديم. سنقوم بتعديله تلقائياً ليتوافق مع بيئة Colab الحديثة ثم نقوم بتثبيت المكتبات.

In [8]:
# Cell 2: Fix requirements and install all dependencies
print("⏳ Modifying requirements.txt for full compatibility...")

# Remove all version constraints to let pip resolve dependencies
!sed -i -E 's/==[0-9.]+//' requirements.txt
# The original repo has a requirement for opencv-python, let's make sure it's headless
!sed -i 's/opencv-python/opencv-python-headless/' requirements.txt


print("✅ requirements.txt has been updated.")
print("\nNew content of requirements.txt:")
!cat requirements.txt

print("\n⏳ Installing required libraries and gdown for Google Drive downloads...")
# Install gdown first
!pip install -q gdown
# Then install from the modified requirements file
!pip install -r requirements.txt -q
print("\n✅ All libraries installed successfully!")

⏳ Modifying requirements.txt for full compatibility...
✅ requirements.txt has been updated.

New content of requirements.txt:
tensorflow
opencv-python-headless
numpy
matplotlib
scikit-learn
tqdm

⏳ Installing required libraries and gdown for Google Drive downloads...

✅ All libraries installed successfully!


### الخطوة الثالثة: تحميل مجموعة البيانات (مصحح)

سنقوم بتحميل البيانات من الرابط المحدث. هذا الكود سيقوم بحذف أي ملف تحميل تالف أو غير مكتمل لضمان تحميل سليم.

In [9]:
# Cell 3: Download the dataset from the alternative Google Drive link
import os

zip_file_path = '../RadDet_dataset.zip' # Using a new name to avoid conflicts
# This is the direct download ID for the dataset from the project's official GitHub page
google_drive_id = '1R4BWGO-aC8c-AY_16A8W2kYp-mclTimS'

if not os.path.exists(zip_file_path):
    print(f"Downloading the dataset from Google Drive... (This may take a few minutes, size is ~556MB)")
    # Use gdown to download from Google Drive
    !gdown --id {google_drive_id} -O {zip_file_path}
else:
    print("Dataset already downloaded.")

# Verify download was successful by checking file size
if os.path.exists(zip_file_path) and os.path.getsize(zip_file_path) > 500 * 1024 * 1024: # Check if > 500MB
    print("✅ Dataset downloaded successfully.")
else:
    print("❌ Download failed or file is corrupted. Please try running the cell again.")

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1R4BWGO-aC8c-AY_16A8W2kYp-mclTimS

but Gdown can't. Please check connections and permissions.
❌ Download failed or file is corrupted. Please try running the cell again.


### الخطوة الرابعة: فك ضغط البيانات

نقوم بفك ضغط ملف البيانات. سيتأكد الكود من وجود الملف قبل محاولة فك ضغطه.

In [10]:
# Cell 4: Unzip the data
import os

data_dir = 'RadDet_data/RadDet-1T-128'
zip_file_path = '../RadDet_dataset.zip' # Use the new file name

if not os.path.exists(data_dir) and os.path.exists(zip_file_path):
    print("Unzipping the data...")
    os.makedirs('RadDet_data', exist_ok=True)
    !unzip -q {zip_file_path} -d RadDet_data
    print("✅ Data unzipped successfully.")
elif os.path.exists(data_dir):
    print("Data directory already exists. Skipping unzip.")
else:
    print("❌ Zip file not found. Please re-run the download step.")

❌ Zip file not found. Please re-run the download step.


### الخطوة الخامسة: استكشاف البيانات

نعرض عينة من البيانات (خريطة Range-Doppler) للتأكد من أن كل شيء تم تحميله وفك ضغطه بشكل صحيح.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Path to a sample radar data file
file_path = 'RadDet_data/RadDet-1T-128/train/seq_000/r_d.npy'

if os.path.exists(file_path):
    print("Displaying a sample Range-Doppler Map...")
    rd_data = np.load(file_path)
    plt.figure(figsize=(8, 6))
    plt.imshow(rd_data, cmap='hot', interpolation='nearest')
    plt.title("Range-Doppler Map")
    plt.colorbar(label="Intensity")
    plt.xlabel("Doppler")
    plt.ylabel("Range")
    plt.show()
else:
    print(f"❌ File not found at {file_path}. Please check the download and unzip steps.")

❌ File not found at RadDet_data/RadDet-1T-128/train/seq_000/r_d.npy. Please check the download and unzip steps.


### الخطوة السادسة: تعديل الإعدادات للتدريب (مصحح)

نقوم بتعديل ملف الإعدادات `RadDet_config.json` لتقليل وقت التدريب (epochs = 5) لأغراض العرض التوضيحي، ونتأكد من حفظ التغييرات.

In [6]:
import json

config_path = 'RadDet_config.json'

print(f"Reading and updating config file: {config_path}")
with open(config_path, 'r') as f:
    config = json.load(f)

original_epochs = config['epochs']
# Modify the number of epochs for a quicker training demonstration
config['epochs'] = 5
print(f"Changing 'epochs' from {original_epochs} to {config['epochs']} for a quick demo.")

# IMPORTANT: Write the changes back to the file
with open(config_path, 'w') as f:
    json.dump(config, f, indent=4)
print("✅ Configuration saved successfully.")

Reading and updating config file: RadDet_config.json


FileNotFoundError: [Errno 2] No such file or directory: 'RadDet_config.json'

### الخطوة السابعة: تدريب النموذج

الآن كل شيء جاهز. نبدأ عملية تدريب النموذج. ستعمل هذه الخلية الآن لـ 5 epochs فقط.

In [ ]:
!python train.py --config_path RadDet_config.json

### الخطوة الثامنة: تقييم النموذج

بعد اكتمال التدريب، نقوم بتقييم أداء النموذج على مجموعة بيانات الاختبار.

In [ ]:
!python eval.py --config_path RadDet_config.json